#**Muyi's Workshop Placer Script**

### How to run:

1. Make sure all the innovation lab data and their schedules, are uploaded in the format shown in "Example Workshop List".

2. Likewise, make sure all the student preference data is uploaded in the format shown in "Example Students List". This can be done auto-magically through a Google Form!

3. For now, **keep STUDENTS_MUST_ATTEND_ALL_SESSIONS variable to False.** Muyi will implement the True aspect later if necessary (not hard, but hopefully unnecessary).

4. For all the other constants in CAPS (*code section 2*) , edit parameter values as necessary. Descriptions to what each variable means are commented next to them.

5. **Hit 'Runtime' at the top and navigate to 'Restart and Run All'. Connect to Google Drive when pop up comes.**

6. Scroll down and enjoy! You can play with the output, it makes tables sometimes. The schedule for each student is created at the bottom and **automatically saved to "Crafted Schedule"** (another sheet in this folder). After making a schedule, **make sure to download it or make a copy if you intend on using it** before running this script again!

### *Other Notes:*
7. One big assumption I made that **each innovation lab only meets once a day**. If not true, update me to rework. On your end, the lab sheet should be changed to show two different sessions on the same day as two different labs named something like: "Artificial Intelligence 1/2" or "Artificial Intelligence Morning/Evening Session".

8. Currently, the script ignores earlier crafted schedules, so if new people sign up via form and need to be put in classes, everyone's schedules will change when the script is re-run. This is why **it's important to make a copy of a finalized schedule.** *(Muyi can implement it so that the script instead builds upon an existing schedule, so update him. It should take about an hour to add that functionality.)*

9. A huge aspect of this is the scheduling! Right now, *you don't get placed into a class if the time overlaps with another you're already in and prefer*. Ask Muyi and he can change it to allow a certain amount of overlap (<= 30 minutes or something like that).

10. Don't delete any code, lol, although I've locally saved a backup. Some of the table outputs have blue buttons next to them that let you see graphs or interact with the tables. Those are fine to play with.

In [1]:
#Workshop Placer Script:
import pandas as pd
import re

from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

**Edit constants below to ensure the right parameters are in place.**

In [2]:
LAB_SHEET_NAME = "Example Workshop List" # The name of the Google Sheet (must be in this folder!) holding the list of innovation lab schedules

STUDENT_SHEET_NAME = "Example Students List" # The name of the Google Sheet (must be in this folder!) holding the list of people attending the conference and their preferences (can be up to infinite unique preferences)

STUDENTS_MUST_ATTEND_ALL_SESSIONS = False # **Keep as False**. What it refers to is if there are innovation labs which meet over both days, do attendees have to go to both?

MAX_DAYS = 2 # The number of days of the conference.

LABS_PER_STUDENT = 2 # The max number of innovation labs sessions an attendee can be in per day

MAX_PREFERENCES_SEARCHED = 3 # The number of preferences each attendee was asked for (and is in sheet, obviously)

Beginning the code stuff. Quick display of the first few entries of each spreadsheets.

In [3]:
#Convert to dataframe
labs = gc.open(LAB_SHEET_NAME).sheet1
students = gc.open(STUDENT_SHEET_NAME).sheet1

# get_all_values gives a list of rows.
lab_rows = labs.get_all_values()
student_rows = students.get_all_values()

labs = pd.DataFrame.from_records(lab_rows)
students = pd.DataFrame.from_records(student_rows)

if not STUDENTS_MUST_ATTEND_ALL_SESSIONS:
  for row in range(1, labs.shape[0]):
    capacity = []
    for day in range(MAX_DAYS):
      capacity.append(labs[1][row])
    labs[1][row] = capacity

labs

,0,1,2,3,4,5,6,7
0,Lab Name,Max Capacity,Meeting Day 1?,Start Time,End Time,Meeting Day 2?,Start Time,End Time
1,Networks and IT,"[2, 2]",TRUE,9:30 AM,11:00 AM,TRUE,11:00 AM,1:00 PM
2,Environment,"[2, 2]",TRUE,9:30 AM,11:00 AM,TRUE,1:30 PM,3:00 PM
3,Finance,"[2, 2]",TRUE,11:30 AM,1:00 PM,TRUE,9:30 AM,11:00 AM
4,International Policy,"[2, 2]",TRUE,11:30 AM,1:00 PM,TRUE,11:30 AM,1:00 PM
5,Fashion,"[2, 2]",TRUE,12:00 PM,2:00 PM,TRUE,9:30 AM,11:00 AM
6,Music,"[2, 2]",TRUE,12:00 PM,2:00 PM,TRUE,2:00 PM,3:30 PM
7,Cybersecurity,"[2, 2]",FALSE,,,TRUE,9:00 AM,12:30 PM
8,Human Resources,"[2, 2]",TRUE,10:00 AM,1:00 PM,FALSE,,
9,Artificial Intelligence,"[2, 2]",TRUE,1:00 PM,2:30 PM,TRUE,12:00 PM,2:00 PM


In [4]:
students

,0,1,2,3,4
0,Student Name,Email,Preference 1,Preference 2,Preference 3
1,Muyi,muyi.aghedo,Artificial Intelligence,Music,Cybersecurity
2,Audrey,audrey.gatera,Healthcare,Music,Human Resources
3,Abi,abi.ndikum,International Policy,Fashion,Finance
4,Grace,grace.dossou,Healthcare,Environment,Finance
5,Fateya,fateya.omer,International Policy,Finance,Cybersecurity
6,Jesse,jesse.okoche,Finance,Fashion,Music


Here's where you can see we start off as a blank schedule. Every new iteration is fair, but random while maximizing preferences.

In [5]:
#Creates a new dataframe to save students and their placements
result = pd.DataFrame(index = students[0])
result[0] = ""
result[0][0] = "Preferences Satisfied"


for labNum in range(LABS_PER_STUDENT):
  for dayIndex in range(MAX_DAYS):
    result[2*(dayIndex + labNum * MAX_DAYS) + 1] = "None"
    result[2*(dayIndex + labNum * MAX_DAYS) + 2] = "Skip"

    result[2*(dayIndex + labNum * MAX_DAYS) + 1][0] = "Lab " + str(labNum + 1) + ", Day " + str(dayIndex + 1)
    result[2*(dayIndex + labNum * MAX_DAYS) + 2][0] = "Day " + str(dayIndex + 1) + " Time"

result

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
Student Name,Preferences Satisfied,"Lab 1, Day 1",Day 1 Time,"Lab 1, Day 2",Day 2 Time,"Lab 2, Day 1",Day 1 Time,"Lab 2, Day 2",Day 2 Time
Muyi,,None,Skip,None,Skip,None,Skip,None,Skip
Audrey,,None,Skip,None,Skip,None,Skip,None,Skip
Abi,,None,Skip,None,Skip,None,Skip,None,Skip
Grace,,None,Skip,None,Skip,None,Skip,None,Skip
Fateya,,None,Skip,None,Skip,None,Skip,None,Skip
Jesse,,None,Skip,None,Skip,None,Skip,None,Skip


**Important Functions!** No touch!

In [6]:
def getClassByName(name, labs):
  return labs[labs[0] == name].iloc[0]

def getClassIdByName(name, labs):
  return labs[labs[0] == name].index.tolist()[0]

def getClassTimesByName(name, labs):
  lab = getClassByName(name, labs)

  timeSheet = []

  #Allows for reading of any number of sessions per lab (on different days)
  for cursor in range(2, labs.shape[1], 3):
    if lab[cursor] == 'TRUE':
      timeSheet.append((lab[cursor + 1], lab[cursor + 2]))
    else:
      timeSheet.append(None)

  return timeSheet

def getClassCapacityByName(name, dayIndex, labs):
  lab = getClassByName(name, labs)
  return int(lab[1][dayIndex])


#Function testing
print(getClassIdByName("Cybersecurity", labs))
print()
print(getClassTimesByName("Cybersecurity", labs))
print(getClassCapacityByName("Cybersecurity", 1, labs))

7

[None, ('9:00 AM', '12:30 PM')]
2


In [7]:
#Gets the preferred class name by a student (rank is 0-indexed)
def getStudentPreferencesByRank(name, rank, students):
  student = students[students[0] == name].iloc[0]
  return student[rank + 2]

def getStudentPreferencesByRankRow(rowIndex, rank, students):
  return students[rank + 2][rowIndex]

#Function testing
print(getStudentPreferencesByRank("Audrey", 0, students)) # Audrey's number one preference
print(getStudentPreferencesByRankRow(2, 0, students)) # Audrey's number one preference

Healthcare
Healthcare


In [8]:
def currentStudentSchedule(rowIndex, result):
  days = ([], [])
  for differentLabColumn in range(1, result.shape[1], 2 * MAX_DAYS): #Correct
    for dI in range(MAX_DAYS):
      #print(result[differentLabColumn + (MAX_DAYS * dI) + 1][0])
      days[dI].append(result[differentLabColumn + (MAX_DAYS * dI) + 1][rowIndex])

  return days

def willOverlapWithSchedule(rowIndex, newTime, result):
  oldSchedule = currentStudentSchedule(rowIndex, result)
  day = [False for x in range(MAX_DAYS)]

  for dI in range(MAX_DAYS):
    overlap = False

    for classTime in oldSchedule[dI]:
      if classTime == "Skip" or classTime is None or newTime[dI] is None:
        continue

      #print(classTime, newTime)
      if not noSessionOverlap(classTime[0], classTime[1], newTime[dI][0], newTime[dI][1]):
        overlap = True
        break;
    day[dI] = overlap

  return day

#Function testing
print(currentStudentSchedule(2, result))
willOverlapWithSchedule(2, [('1:00 PM', '3:00 PM'), ('8:00 AM', '9:00 AM')], result)

(['Skip', 'Skip'], ['Skip', 'Skip'])


[False, False]

In [9]:
def timeStringToValue(time):
  time = time.upper()
  hourMinZone = re.split(":|\s", time)
  hourMinZone[0] = int(hourMinZone[0]) % 12
  hourMinZone[1] = int(hourMinZone[1]) / 60

  if (hourMinZone[2] == 'PM'):
    hourMinZone[2] = 12
  else:
    hourMinZone[2] = 0

  return sum(hourMinZone)

#Function testing
print(timeStringToValue("11:30 pm"))

23.5


In [10]:
def noSessionOverlap(start1, end1, start2, end2):
  #Turn time strings to easy-to-use numbers, make sure overnight events are accounted for
  start1 = timeStringToValue(start1)
  end1 = timeStringToValue(end1)

  if start1 > end1:
    end1 = end1 + 24

  start2 = timeStringToValue(start2)
  end2 = timeStringToValue(end2)

  if start2 > end2:
    end2 = end2 + 24

  # No overlap means start1 < end1 <= start2 < end2 OR start2 < end2 <= start1 < end1
  if (end1 <= start2 or end2 <= start1):
    return True
  #else:
  return False

#Function testing
print(noSessionOverlap("11:00 am", "12:30 pm", "1:00 pm", "2:30 pm"))
print(noSessionOverlap("1:00 pm", "2:30 pm", "11:00 am", "12:30 pm"))
print(noSessionOverlap("12:00 pm", "2:30 pm", "11:00 am", "12:30 pm"))
print(noSessionOverlap("1:00 pm", "2:30 am", "11:00 am", "1:30 am"))

True
True
False
False


In [11]:
#Again, 0 is the highest preference
def studentPreferenceForClass(studentRowIndex, className, students):
  for pref in range(2, students.shape[1]):
    if students[pref][studentRowIndex] == className:
      return pref - 2
  return False;

#Checks if a student can register for a class
def isClassAvailable(studentRowIndex, className, students, result, dayIndex):
  assignedClasses = []
  for labNum in range(LABS_PER_STUDENT):
    assignedClasses.append(result[2*(dayIndex + labNum * MAX_DAYS) + 1][studentRowIndex])

  #is student already in the lab?
  if className in assignedClasses:
    #print(students[0][studentRowIndex] + " is already in " + className + " on day " + str(dayIndex + 1))
    return False

  #is lab full?
  if getClassCapacityByName(className, dayIndex, labs) <= 0:
    #print(className + " too full to add " + students[0][studentRowIndex])
    return False

  labTimes = getClassTimesByName(className, labs)
  #Does the class exist on said day?
  if labTimes[dayIndex] is None:
    #print(className + " doesn't hold on Day " + str(dayIndex + 1))
    return False

  #Schedule conflict on dayIndex?
  daytupleOverlap = willOverlapWithSchedule(studentRowIndex, labTimes, result)

  #There is an overlap on said day
  if daytupleOverlap[dayIndex]:
    #print(students[0][studentRowIndex] + " has a schedule overlap with " + className + " on day " + str(dayIndex + 1))
    return False

  #There are no conflicts
  return True

#Lists all classes on a particular day that student can still register for
def allAvailableClasses(studentRowIndex, students, result, labs, dayIndex):
  availableClasses = []
  for classIndex in range(1, labs.shape[0]):
    className = labs[0][classIndex]
    if isClassAvailable(studentRowIndex, className, students, result, dayIndex):
      availableClasses.append(className)
  return availableClasses

print(studentPreferenceForClass(1, "Music", students)) # Music is Muyi's second choice
print(isClassAvailable(1, "Artificial Intelligence", students, result, 1)) # AI is an available class for Muyi
print(allAvailableClasses(2, students, result, labs, 0)) # Cybersecurity is not available on day 1 for Audrey

1
True
['Networks and IT', 'Environment', 'Finance', 'International Policy', 'Fashion', 'Music', 'Human Resources', 'Artificial Intelligence', 'Healthcare']


**Beginning of the sorting algorithm!**

In [12]:
# Important for 'fairness'
import random

# Makes a list from 1 - the number of students and orders it randomly
choiceOrder = [*range(1, students.shape[0])]
random.shuffle(choiceOrder)

In [13]:
#Sorting by day. Each day, everyone chooses LABS_PER_STUDENT number of lab sessions according to their preferences.
for dayIndex in range(0, 2):
  for labNum in range(LABS_PER_STUDENT):
    choiceOrder.reverse()

    for studentId in choiceOrder:
      studentName = students[0][studentId] #Helpful for debugging

      #Look through this student's preferred labs
      for pref in range(MAX_PREFERENCES_SEARCHED):
        preferredLab = getStudentPreferencesByRank(studentName, pref, students)
        labTime = getClassTimesByName(preferredLab, labs)
        labIndex = getClassIdByName(preferredLab, labs)

        assigned = False

        #print(dayIndex, studentName, pref, preferredLab, isClassAvailable(studentId, preferredLab, students, result, dayIndex), result[2*(dayIndex + labNum * MAX_DAYS) + 1][0])

        #Check if the class is available for student
        if isClassAvailable(studentId, preferredLab, students, result, dayIndex):
          #Add class to student schedule, subtract one from class capacity, and add preference number to result[0][studentId]
          #print(studentName, dayIndex, pref, preferredLab, result[2*(dayIndex + labNum * MAX_DAYS) + 1][0])
          result[2*(dayIndex + labNum * MAX_DAYS) + 1][studentId] = preferredLab
          result[2*(dayIndex + labNum * MAX_DAYS) + 2][studentId] = labTime[dayIndex]

          #Subtract from class capacity
          #print(getClassCapacityByName(preferredLab, dayIndex, labs), labs[1][labIndex])
          labs[1][labIndex][dayIndex] = int(labs[1][labIndex][dayIndex]) - 1
          #print(getClassCapacityByName(preferredLab, dayIndex, labs), labs[1][labIndex])

          #You put a class, so chill
          result[0][studentId] += str(pref) + "|"
          assigned = True
          break

      #Immediately assigns random available lab to student. Honestly, though because of other's preferences we should probably revisit the Nones/Skips after getting through everyone...
      #Remediate later
      #if not assigned:
      #  If we've gotten here then student has no labs amongst their preferences, so assign a random available lab
      #  #availableLabs = allAvailableClasses(studentId, students, result, labs, dayIndex)

      #  #Assign here
      #  #if len(availableLabs) > 0:
      #    assignedLab = random.choice(availableLabs)
      #    labTime = getClassTimesByName(assignedLab, labs)
      #    labIndex = getClassIdByName(assignedLab, labs)
      #    print(dayIndex, studentName, assignedLab, isClassAvailable(studentId, assignedLab, students, result, dayIndex), result[2*(dayIndex + labNum * MAX_DAYS) + 1][0])
      #
      #    #Assignment
      #    result[2*(dayIndex + labNum * MAX_DAYS) + 1][studentId] = assignedLab
      #    result[2*(dayIndex + labNum * MAX_DAYS) + 2][studentId] = labTime[dayIndex]
      #
      #    #Capacity - 1
      #    labs[1][labIndex][dayIndex] = int(labs[1][labIndex][dayIndex]) - 1
      #
      #    #Preference log update
      #    result[0][studentId] += "NA |"
      #
      #
      #  #else:
      #  #Ain't no labs left today!

result

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
Student Name,Preferences Satisfied,"Lab 1, Day 1",Day 1 Time,"Lab 1, Day 2",Day 2 Time,"Lab 2, Day 1",Day 1 Time,"Lab 2, Day 2",Day 2 Time
Muyi,0|0|1|,Artificial Intelligence,"(1:00 PM, 2:30 PM)",Artificial Intelligence,"(12:00 PM, 2:00 PM)",None,Skip,Music,"(2:00 PM, 3:30 PM)"
Audrey,0|1|0|,Healthcare,"(2:00 PM, 3:30 PM)",Healthcare,"(2:30 PM, 4:00 PM)",Music,"(12:00 PM, 2:00 PM)",None,Skip
Abi,0|0|1|,International Policy,"(11:30 AM, 1:00 PM)",International Policy,"(11:30 AM, 1:00 PM)",None,Skip,Fashion,"(9:30 AM, 11:00 AM)"
Grace,0|1|0|,Healthcare,"(2:00 PM, 3:30 PM)",Healthcare,"(2:30 PM, 4:00 PM)",Environment,"(9:30 AM, 11:00 AM)",None,Skip
Fateya,0|0|1|,International Policy,"(11:30 AM, 1:00 PM)",International Policy,"(11:30 AM, 1:00 PM)",None,Skip,Finance,"(9:30 AM, 11:00 AM)"
Jesse,0|0|2|,Finance,"(11:30 AM, 1:00 PM)",Finance,"(9:30 AM, 11:00 AM)",None,Skip,Music,"(2:00 PM, 3:30 PM)"


In [14]:
#Checking for skips and Nones (at this point, everyone has had their preferences matched as much as possible, so just add random labs to the session that were skipped)
for dayIndex in range(0, 2):
  for labNum in range(LABS_PER_STUDENT):
    for studentId in range(1, result.shape[0]):
      if result[2*(dayIndex + labNum * MAX_DAYS) + 1][studentId] == 'None':

        #print(students[0][studentId], result[2*(dayIndex + labNum * MAX_DAYS) + 1][0])

        availableLabs = allAvailableClasses(studentId, students, result, labs, dayIndex)

        #Assign here
        if len(availableLabs) > 0:
          assignedLab = random.choice(availableLabs)
          labTime = getClassTimesByName(assignedLab, labs)
          labIndex = getClassIdByName(assignedLab, labs)
          #print(dayIndex, studentName, assignedLab, isClassAvailable(studentId, assignedLab, students, result, dayIndex), result[2*(dayIndex + labNum * MAX_DAYS) + 1][0])

          #Assignment
          result[2*(dayIndex + labNum * MAX_DAYS) + 1][studentId] = assignedLab
          result[2*(dayIndex + labNum * MAX_DAYS) + 2][studentId] = labTime[dayIndex]

          #Capacity - 1
          labs[1][labIndex][dayIndex] = int(labs[1][labIndex][dayIndex]) - 1

          #Preference log update
          result[0][studentId] += "NA |"


        #else:
        #Ain't no labs left today!

result[0] = result.index

**Here's your final result!!!**

In [15]:
result

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
Student Name,Student Name,"Lab 1, Day 1",Day 1 Time,"Lab 1, Day 2",Day 2 Time,"Lab 2, Day 1",Day 1 Time,"Lab 2, Day 2",Day 2 Time
Muyi,Muyi,Artificial Intelligence,"(1:00 PM, 2:30 PM)",Artificial Intelligence,"(12:00 PM, 2:00 PM)",Environment,"(9:30 AM, 11:00 AM)",Music,"(2:00 PM, 3:30 PM)"
Audrey,Audrey,Healthcare,"(2:00 PM, 3:30 PM)",Healthcare,"(2:30 PM, 4:00 PM)",Music,"(12:00 PM, 2:00 PM)",Artificial Intelligence,"(12:00 PM, 2:00 PM)"
Abi,Abi,International Policy,"(11:30 AM, 1:00 PM)",International Policy,"(11:30 AM, 1:00 PM)",Artificial Intelligence,"(1:00 PM, 2:30 PM)",Fashion,"(9:30 AM, 11:00 AM)"
Grace,Grace,Healthcare,"(2:00 PM, 3:30 PM)",Healthcare,"(2:30 PM, 4:00 PM)",Environment,"(9:30 AM, 11:00 AM)",Networks and IT,"(11:00 AM, 1:00 PM)"
Fateya,Fateya,International Policy,"(11:30 AM, 1:00 PM)",International Policy,"(11:30 AM, 1:00 PM)",Networks and IT,"(9:30 AM, 11:00 AM)",Finance,"(9:30 AM, 11:00 AM)"
Jesse,Jesse,Finance,"(11:30 AM, 1:00 PM)",Finance,"(9:30 AM, 11:00 AM)",Networks and IT,"(9:30 AM, 11:00 AM)",Music,"(2:00 PM, 3:30 PM)"


Saving to "Crafted Schedule" below, replacing whatever was there before.

In [16]:
#Save result as new spreadsheet
import gspread_dataframe as gd


ws = gc.open("Crafted Schedule").worksheet("Sheet1")
ws.clear()
gd.set_with_dataframe(ws, result)